<a href="https://colab.research.google.com/github/ymann/Machine-Comprehension-Cloze-QA/blob/master/Non_Deep_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! wget http://seas.upenn.edu/~branlin/cnn.tgz 
! tar -xzf cnn.tgz

--2019-04-23 01:01:39--  http://seas.upenn.edu/~branlin/cnn.tgz
Resolving seas.upenn.edu (seas.upenn.edu)... 158.130.68.91
Connecting to seas.upenn.edu (seas.upenn.edu)|158.130.68.91|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.seas.upenn.edu/~branlin/cnn.tgz [following]
--2019-04-23 01:01:39--  https://www.seas.upenn.edu/~branlin/cnn.tgz
Resolving www.seas.upenn.edu (www.seas.upenn.edu)... 158.130.68.91, 2607:f470:8:64:5ea5::9
Connecting to www.seas.upenn.edu (www.seas.upenn.edu)|158.130.68.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217447312 (207M) [application/x-gzip]
Saving to: ‘cnn.tgz’

cnn.tgz             100%[===================>] 207.37M  18.5MB/s    in 12s     

2019-04-23 01:01:51 (17.7 MB/s) - ‘cnn.tgz’ saved [217447312/217447312]



In [0]:
!git clone https://github.com/cis700/hw1-release.git
!mv hw1-release/helper.py .
!rm -rf hw1-release/
!rm -rf hw1/

Cloning into 'hw1-release'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 26 (delta 6), reused 23 (delta 6), pack-reused 0
Unpacking objects: 100% (26/26), done.


In [0]:
LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

!if [ -f ngrok ] ; then echo "Ngrok already installed" ; else wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip > /dev/null 2>&1 && unzip ngrok-stable-linux-amd64.zip > /dev/null 2>&1 ; fi


In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print('Tensorboard Link: ' +str(json.load(sys.stdin)['tunnels'][0]['public_url']))"

Tensorboard Link: https://2ab70209.ngrok.io


In [0]:
import torch.nn as nn
import torch.utils.data
import os
from collections import Counter
from nltk import ngrams

class QuestionDataset(torch.utils.data.Dataset):
  
  def __init__(self, root_dir):
    self.root_dir = root_dir
    self.files = os.listdir(os.path.join(os.getcwd(), root_dir))
    
  def __len__(self):
    return len(self.files)
  
  def __getitem__(self, idx):
    with open(os.path.join(self.root_dir, self.files[idx]), 'r') as f:
      s = f.read()
    
    lines = s.split('\n')
    doc = lines[2]
    tokens = doc.split()
    query = lines[4]
    query_tokens = query.split()
    correct_entity = lines[6]
    entities = [tuple(i.split(":")) for i in lines[8:]]
    return (tokens, query, correct_entity, entities)
  
class LogisticRegression(nn.Module):
  
  def __init__(self, num_classes=8):
    super(LogisticRegression, self).__init__()
    
    self.fc = nn.Sequential(
        nn.Linear(num_classes, 1),
        nn.Sigmoid()
    )
    
  def forward(self, x):
    return self.fc(x)
  
# We extract an entity e from the document d and query q based on the following features:
# 1. how many times e appears in d
# 2. where e is first mentioned in d
# 3. proximity match: whether [e, __, w] appears in d, where w is the word succeeding @placeholder and __ is 0 to 2 words. Feature will be the number of spaces
# 4. proximity: whether [w, __, e] appears in d, where w is word preceding @placeholder. Feature will be number of spaces

def extract_features(tokens, query, correct_entity, entities):
  '''
  entity should be of the form (@entity<n>, word) as a tuple
  '''
  
  ctr = Counter(tokens)
  placeholder_index = query.index('@placeholder')
  word_before = query[placeholder_index - 1] if placeholder_index > 0 else None
  word_after = query[placeholder_index + 1] if placeholder_index < len(query) else None
  
  # Generate n grams for the passage, where n ranges from 2 to 4.
  bigrams = [(i, 1) for i in ngrams(tokens, 2)]
  trigrams = [(i, 2) for i in ngrams(tokens, 3)]
  quadgrams = [(i, 3) for i in ngrams(tokens, 4)]
  grams = bigrams + trigrams + quadgrams
  
  features = []
  for e in entities:
    entity_count = ctr[e[0]]
    try:
      first_mention = tokens.index(e[0])
    except ValueError:
      first_mention = -1
    prox_before, prox_after = 0, 0 # represents if no proximity match
    for gram in grams:
      if gram[0][0] == word_before and gram[0][-1] == e[0]:
        prox_before = gram[1]
        break
    for gram in grams:
      if gram[0][0] == e[0] and gram[0][-1] == word_after:
        prox_after = gram[1]
        break
    features.append((torch.Tensor([entity_count, first_mention, prox_before, prox_after]), torch.Tensor([1 if correct_entity == e[0] else 0])))
  
  return features

In [0]:
from torch.utils.data import DataLoader
import torch.nn as nn

class AuxiliaryQuestionsDataset(torch.utils.data.Dataset):
  
  def __init__(self, subset='train'):
    if subset == 'train':
      dataset = QuestionDataset("cnn/questions/training")
    elif subset == 'test':  
      dataset = QuestionDataset("cnn/questions/test")
    elif subset == 'validation':
      dataset = QuestionDataset("cnn/questions/validation")
    else:
      raise ValueError("Invalid arguments passed; expected train or test.")
      
    self.data = []
    for i, (tokens, query, correct_entity, entities) in enumerate(dataset):
      if i % 10000 == 0:
        print(i)
      self.data.extend(extract_features(tokens, query, correct_entity, entities))
  
  def __len__(self):
    return len(self.data)
  
  def __getitem__(self, idx):
    return self.data[idx]
  
  
train_data = AuxiliaryQuestionsDataset(subset='train')

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000


In [0]:
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
from torch import Tensor as tensor

class AuxiliaryQuestionsDataset(torch.utils.data.Dataset):
  
  def __init__(self, subset='train'):
#     if subset == 'train':
#       dataset = QuestionDataset("cnn/questions/training")
#     elif subset == 'test':  
#       dataset = QuestionDataset("cnn/questions/test")
#     elif subset == 'validation':
#       dataset = QuestionDataset("cnn/questions/validation")
#     else:
#       raise ValueError("Invalid arguments passed; expected train or test.")
      
    with open('logregfeatures.txt', 'r') as f:
      self.lines = f.readlines()
  def __len__(self):
    return len(self.lines)
  
  def __getitem__(self, idx):
    tup = eval(self.lines[idx])
    return tup
  
  
train_data = AuxiliaryQuestionsDataset(subset='train')

In [0]:
train_data

In [0]:
# http://pytorch.org/
# from os.path import exists
# from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
# platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
# cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.cd\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
# accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
# !pip3 install https://download.pytorch.org/whl/cu100/torch-1.0.1-cp36-cp36m-linux_x86_64.whl
# !pip3 install torchvision
  
import torch
device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
# Code referenced from https://gist.github.com/gyglim/1f8dfb1b5c82627ae3efcfbbadb9f514
import tensorflow as tf
import numpy as np
import scipy.misc 
try:
    from StringIO import StringIO  # Python 2.7
except ImportError:
    from io import BytesIO         # Python 3.x


class Logger(object):
    
    def __init__(self, log_dir):
        """Create a summary writer logging to log_dir."""
        self.writer = tf.summary.FileWriter(log_dir)

    def scalar_summary(self, tag, value, step):
        """Log a scalar variable."""
        summary = tf.Summary(value=[tf.Summary.Value(tag=tag, simple_value=value)])
        self.writer.add_summary(summary, step)

    def image_summary(self, tag, images, step):
        """Log a list of images."""

        img_summaries = []
        for i, img in enumerate(images):
            # Write the image to a string
            try:
                s = StringIO()
            except:
                s = BytesIO()
            scipy.misc.toimage(img).save(s, format="png")

            # Create an Image object
            img_sum = tf.Summary.Image(encoded_image_string=s.getvalue(),
                                       height=img.shape[0],
                                       width=img.shape[1])
            # Create a Summary value
            img_summaries.append(tf.Summary.Value(tag='%s/%d' % (tag, i), image=img_sum))

        # Create and write Summary
        summary = tf.Summary(value=img_summaries)
        self.writer.add_summary(summary, step)
        
    def histo_summary(self, tag, values, step, bins=1000):
        """Log a histogram of the tensor of values."""

        # Create a histogram using numpy
        counts, bin_edges = np.histogram(values, bins=bins)

        # Fill the fields of the histogram proto
        hist = tf.HistogramProto()
        hist.min = float(np.min(values))
        hist.max = float(np.max(values))
        hist.num = int(np.prod(values.shape))
        hist.sum = float(np.sum(values))
        hist.sum_squares = float(np.sum(values**2))

        # Drop the start of the first bin
        bin_edges = bin_edges[1:]

        # Add bin edges and counts
        for edge in bin_edges:
            hist.bucket_limit.append(edge)
        for c in counts:
            hist.bucket.append(c)

        # Create and write Summary
        summary = tf.Summary(value=[tf.Summary.Value(tag=tag, histo=hist)])
        self.writer.add_summary(summary, step)
        self.writer.flush()


In [0]:
import torch
import datetime
logger = Logger('./logs/run_' + str(datetime.datetime.now()))

dataloader = DataLoader(train_data, batch_size=200, shuffle=True)
net = LogisticRegression(num_classes=4).to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.0001)

num_epochs = 5
step = 0

for epoch in range(num_epochs):
  print("Running epoch %d" % epoch)
  data_iter = iter(dataloader)
  for batch_features, batch_labels in data_iter:
    batch_features = batch_features.to(device)
    batch_labels = batch_labels.to(device)
    output = net(batch_features)
    
    loss = criterion(output, batch_labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    logger.scalar_summary("loss", loss.item(), step+1)
    step += 1
    

Running epoch 0


In [0]:
other_dataset = QuestionDataset('cnn/questions/validation')

total_correct, total = 0, 0
with torch.no_grad():
  for example in other_dataset:
    total += 1
    ex = extract_features(*example)
    features, label = [torch.stack(list(i)) for i in zip(*ex)]
    print(features)
    features = features.to(device)
    label = label.to(device)
    output = net(features)
    
    _, argmax = torch.max(output, 0)
    _, argmax2 = torch.max(label, 0)
    
    if (argmax == argmax2):
      total_correct += 1
    
    

In [0]:
print (total_correct / total)

0.290519877675841
